#Production

#import

In [1]:
#общий
import pandas as pd  #работа с датасетами
import numpy as np # работа с массивами

#preprocessing
from sklearn.preprocessing import LabelEncoder , MinMaxScaler #кодирвоание категориальныех признаков
from sklearn.model_selection import (train_test_split, #разделение на обучающую и тестовую выборки
                                     GridSearchCV #подбор гиперпараметров мтеодом сетки
                                     )

#ML classification
from sklearn.neighbors import KNeighborsClassifier #метод ближайших соседей

#для сохранения и загрузки моделей
import pickle

#Загрузка данных

In [2]:
!git clone https://github.com/Lasttrader/DA_repo

Cloning into 'DA_repo'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 69 (delta 3), reused 1 (delta 0), pack-reused 58 (from 1)
Receiving objects: 100% (69/69), 31.82 MiB | 10.43 MiB/s, done.
Resolving deltas: 100% (16/16), done.
Updating files: 100% (26/26), done.


In [3]:
path = '/content/DA_repo/bank.csv'

In [4]:
data = pd.read_csv(path, sep = ';')
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   object
dtypes: int64(7), object(10)
memory usage: 600.6+ KB


#Работа с признаками

##категориальные

In [6]:
cat_cols = data.describe(include = 'object').columns.to_list() #список с категориальными признаками
cat_data = data[cat_cols]
cat_data.head()

,job,marital,education,default,housing,loan,contact,month,poutcome,y
0,unemployed,married,primary,no,no,no,cellular,oct,unknown,no
1,services,married,secondary,no,yes,yes,cellular,may,failure,no
2,management,single,tertiary,no,yes,no,cellular,apr,failure,no
3,management,married,tertiary,no,yes,yes,unknown,jun,unknown,no
4,blue-collar,married,secondary,no,yes,no,unknown,may,unknown,no


In [7]:
le = LabelEncoder()
for column in cat_cols:
  cat_data[column + '_LE'] = le.fit_transform(cat_data[column])
  with open(f'{column}_LE.pkl', 'wb') as f:
    pickle.dump(le, f)

cat_data.head()

<ipython-input-7-2e23250f7f79>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_data[column + '_LE'] = le.fit_transform(cat_data[column])
<ipython-input-7-2e23250f7f79>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_data[column + '_LE'] = le.fit_transform(cat_data[column])
<ipython-input-7-2e23250f7f79>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

,job,marital,education,default,housing,loan,contact,month,poutcome,y,job_LE,marital_LE,education_LE,default_LE,housing_LE,loan_LE,contact_LE,month_LE,poutcome_LE,y_LE
0,unemployed,married,primary,no,no,no,cellular,oct,unknown,no,10,1,0,0,0,0,0,10,3,0
1,services,married,secondary,no,yes,yes,cellular,may,failure,no,7,1,1,0,1,1,0,8,0,0
2,management,single,tertiary,no,yes,no,cellular,apr,failure,no,4,2,2,0,1,0,0,0,0,0
3,management,married,tertiary,no,yes,yes,unknown,jun,unknown,no,4,1,2,0,1,1,2,6,3,0
4,blue-collar,married,secondary,no,yes,no,unknown,may,unknown,no,1,1,1,0,1,0,2,8,3,0


In [8]:
cat_data.drop(cat_cols, axis = 1, inplace = True)
cat_data.head()

,job_LE,marital_LE,education_LE,default_LE,housing_LE,loan_LE,contact_LE,month_LE,poutcome_LE,y_LE
0,10,1,0,0,0,0,0,10,3,0
1,7,1,1,0,1,1,0,8,0,0
2,4,2,2,0,1,0,0,0,0,0
3,4,1,2,0,1,1,2,6,3,0
4,1,1,1,0,1,0,2,8,3,0


##Числовые

In [9]:
num_cols = data.describe().columns.to_list() #формируем список ТОЛЬКО с числовыми признаками
num_data = data[num_cols] #делаем подвыборку и формируем новый датафрейм
num_data.head()

,age,balance,day,duration,campaign,pdays,previous
0,30,1787,19,79,1,-1,0
1,33,4789,11,220,1,339,4
2,35,1350,16,185,1,330,1
3,30,1476,3,199,4,-1,0
4,59,0,5,226,1,-1,0


In [10]:
'''
#это код который удалит колонку, у которой срелняя равна 0

for column in num_data.columns.to_list():
  if num_data[column].mean == 0:
    num_data.drop(column, axis = 1, inplace = True)
'''

'\n#это код который удалит колонку, у которой срелняя равна 0\n\nfor column in num_data.columns.to_list():\n  if num_data[column].mean == 0:\n    num_data.drop(column, axis = 1, inplace = True)\n'

##итоговый датасет

In [11]:
dataset = pd.concat([cat_data, num_data], axis = 1)
dataset.head()

,job_LE,marital_LE,education_LE,default_LE,housing_LE,loan_LE,contact_LE,month_LE,poutcome_LE,y_LE,age,balance,day,duration,campaign,pdays,previous
0,10,1,0,0,0,0,0,10,3,0,30,1787,19,79,1,-1,0
1,7,1,1,0,1,1,0,8,0,0,33,4789,11,220,1,339,4
2,4,2,2,0,1,0,0,0,0,0,35,1350,16,185,1,330,1
3,4,1,2,0,1,1,2,6,3,0,30,1476,3,199,4,-1,0
4,1,1,1,0,1,0,2,8,3,0,59,0,5,226,1,-1,0


#X,y

In [12]:
y = np.array(dataset['y_LE'])
X = np.array(dataset.drop('y_LE', axis = 1))
print(y[:2])
print(X[:2])

[0 0]
[[  10    1    0    0    0    0    0   10    3   30 1787   19   79    1
    -1    0]
 [   7    1    1    0    1    1    0    8    0   33 4789   11  220    1
   339    4]]


#Scaler

In [13]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

print(X[:2])
print(X_scaled[:2])

#сохраняем модель
with open('num_scaler.pkl', 'wb') as f:
  pickle.dump(scaler, f)

[[  10    1    0    0    0    0    0   10    3   30 1787   19   79    1
    -1    0]
 [   7    1    1    0    1    1    0    8    0   33 4789   11  220    1
   339    4]]
[[0.90909091 0.5        0.         0.         0.         0.
  0.         0.90909091 1.         0.16176471 0.06845546 0.6
  0.02482622 0.         0.         0.        ]
 [0.63636364 0.5        0.33333333 0.         1.         1.
  0.         0.72727273 0.         0.20588235 0.10875022 0.33333333
  0.0714995  0.         0.38990826 0.16      ]]


#train_test_Split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y,
                                                    test_size = 0.2,
                                                    shuffle = True,
                                                    random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3616, 16)
(905, 16)
(3616,)
(905,)


#ML

In [15]:
kNN = KNeighborsClassifier(n_neighbors=5)
kNN.fit(X_train, y_train)

KNeighborsClassifier()

In [16]:
y_pred = kNN.predict(X_test[:5])

print(y_pred)
print(y_test[:5])

[0 1 0 0 0]
[0 0 0 0 0]


#Сохраняем модель

In [17]:
#сохраняем модель
with open('kNN.pkl', 'wb') as f:
  pickle.dump(kNN, f)

#Загружаем модель из файла

In [18]:
with open('/content/kNN.pkl', 'rb') as f:
  loaded_kNN = pickle.load(f)

In [19]:
print(kNN.predict(X_test[:10]))

print(loaded_kNN.predict(X_test[:10]))

[0 1 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0]


#BaseAPP

In [20]:
#import


#load_models


#get_data


#preprocessing
##categorical

##num

##объединить категориальные и числовые (в том же порядке, как и при обучении)

#scaler

#predict

#result


In [ ]:
#import
import pickle 
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
print('import success')

#load_models
#le
job_LE = pickle.load(open('../180824_prod/models/job_LE.pkl', 'rb'))
marital_LE = pickle.load(open('../180824_prod/models/marital_LE.pkl', 'rb')) 	
education_LE = pickle.load(open('../180824_prod/models/education_LE.pkl', 'rb'))	
default_LE = pickle.load(open('../180824_prod/models/default_LE.pkl', 'rb')) 	
housing_LE = pickle.load(open('../180824_prod/models/housing_LE.pkl', 'rb'))	
loan_LE	= pickle.load(open('../180824_prod/models/loan_LE.pkl', 'rb'))
contact_LE	= pickle.load(open('../180824_prod/models/contact_LE.pkl', 'rb'))
month_LE	= pickle.load(open('../180824_prod/models/month_LE.pkl', 'rb'))
poutcome_LE	= pickle.load(open('../180824_prod/models/poutcome_LE.pkl', 'rb'))
#predict le
y_LE = pickle.load(open('../180824_prod/models/y_LE.pkl', 'rb'))
#scaler
num_scaler = pickle.load(open('../180824_prod/models/num_scaler.pkl', 'rb'))
#ML models
kNN = pickle.load(open('../180824_prod/models/kNN.pkl', 'rb'))

#get_data
#вводим с клавиатуры
# #cat_cols
# job	= input()
# marital	= input()
# education	= input()
# default	= input()
# housing= input()
# loan	= input()
# contact= input()
# month= input()
# poutcome= input()

# #num_cols
# age	= float(input())
# balance	= float(input())
# day	= float(input())
# duration = float(input())
# campaign = float(input())
# pdays	 = float(input())
# previous = float(input())



#preprocessing
##categorical
X_cat_from_keyboard = ['unemployed',
                       'married',
                       'primary',
                       'no',
                       'no',
                       'yes',
                       'cellular',
                       'may',
                       'failure'
                        ]
# print(X_cat_from_keyboard)
le_list = [job_LE,	
           marital_LE,	
           education_LE,	
           default_LE,
           housing_LE,
           loan_LE,
           contact_LE,
           month_LE,
           poutcome_LE]

X_le_list = [] #под закодированные признаки
for i in range(len(X_cat_from_keyboard)):
    x_cat = le_list[i].transform([X_cat_from_keyboard[i]])[0]
    # print(x_cat)
    X_le_list.append(x_cat)
print('X_cat_le:', X_le_list)

##num
X_nums_from_keyboard =[30,
                       1787,
                       16,
                       199,
                       4,
                       330,
                       0]
print('X_nums', X_nums_from_keyboard)

##объединить категориальные и числовые (в том же порядке, как и при обучении)
X = []
X.extend(X_le_list)
X.extend(X_nums_from_keyboard)
print('X:', X)

#scaler
X_scaled = num_scaler.transform([X])
print('X_scaled:', X_scaled)

#predict
prediction = kNN.predict(X_scaled)
print(prediction)

#result
result = y_LE.inverse_transform(prediction)
print('Выдавать клиентский договор? Ответ - ', result)